# Projekt 2  Analýza cen bydlení v Brně 



# Sběr a čištění dat


1) Ceny bytů v Brně (prodej i pronájem) --> csv, web scraping

2) Výstavba nových bytů v Brně --> csv

3) Pohyb obyvatelstva (přirozený přírůstek, migrační saldo) v Brně --> API, csv

4) HDP a ČDDD na obyvatele v Jihomoravském kraji --> excel

5) Průměrná mzda v Brně --> csv

6) Sazby hypoték --> web scraping


## 1) Ceny bytů v Brně (prodej i pronájem)

In [1]:
# stazene csv z https://data.brno.cz/rust-cen-nemovitosti-v-brne
# data po cerven 2020

import pandas as pd
import os 


dir_path = os.getcwd() + '\ceny bytu'

df_all_list = []
for filename in os.listdir(dir_path):
    if filename.endswith('.csv'):
        new_df = pd.read_csv(os.path.join(dir_path, filename), index_col=[0])
        new_df.index.names = ['datum']
        if 'prodeje' in filename and 'm2' in filename:
            new_df.rename(columns= {new_df.columns[0]:'prodej (cena m2)'}, inplace=True)
        elif 'prodeje' in filename and 'm2' not in filename: 
            new_df.rename(columns= {new_df.columns[0]:'prodej (cena bytu)'}, inplace=True)
        elif 'pronajem' in filename and 'm2' in filename:
            new_df.rename(columns= {new_df.columns[0]:'pronajem (cena m2)'}, inplace=True)
        elif 'pronajem' in filename and 'm2' not in filename:
            new_df.rename(columns= {new_df.columns[0]:'pronajem (cena bytu)'}, inplace=True)
        df_all_list.append(new_df)

df_ceny = df_all_list[0]
for i in range(1,4):
   df_ceny = df_ceny.merge(df_all_list[i], how ='outer', on = 'datum')

for col in df_ceny.columns:
    if 'Průměrná' in col:
        df_ceny.drop(columns = [col], inplace = True)
    else:
       continue
    
df_ceny.reset_index(inplace=True)    


In [2]:
# doplneni aktualnich dat pomoci scrapingu ze webu realitymix (chybi cervenec a srpen)

import requests as rq
from bs4 import BeautifulSoup as bs


urls = ['https://realitymix.cz/statistika-nemovitosti/byty-prodej-prumerna-cena-bytu.html',
'https://realitymix.cz/statistika-nemovitosti/byty-pronajem-prumerna-cena-pronajmu-za-mesic.html',
'https://realitymix.cz/statistika-nemovitosti/byty-prodej-prumerna-cena-za-1m2-bytu.html',
'https://realitymix.cz/statistika-nemovitosti/byty-pronajem-prumerna-cena-pronajmu-1m2-mesic.html',
]

def new_cols_for_df(pages):
    cols_dict = {'datum': ['září 2020', 'říjen 2020', 'listopad 2020']}
    for page in pages:
        raw_data = get_data(page)
        data = clean_data(raw_data)
        name = get_name_col(page)
        cols_dict[name] = data
    return cols_dict

def get_data(page):
    r = rq.get(page)
    soup = bs(r.text, 'html.parser')
    prices = soup.find('td', string='Brno').find_next_siblings()[1:4]
    return prices

def clean_data(list):
    new_list = [int(i.string.replace('\xa0', '')) for i in list]
    return new_list

def get_name_col(string):
    if 'bytu' in string:
        if 'm2' in string:
            return 'prodej (cena m2)'
        else:
            return 'prodej (cena bytu)'
    elif 'pronaj' in string:
        if 'm2' in string:
            return 'pronajem (cena m2)'
        else:
            return 'pronajem (cena bytu)'

df2 = pd.DataFrame(new_cols_for_df(urls))
df_ceny = pd.concat([df_ceny, df2], ignore_index = True)


In [3]:
#doplneni chybejicich dvou mesicu a spojeni celeho df
#doplneny priblizne hodnoty podle grafu na realitymix (v srpnu byly prodeje nadprumerne, proto neni doplnen prumer apod.)


lines = pd.DataFrame({'datum': ['červenec 2020', 'srpen 2020'],
        'prodej (cena bytu)': [5340000, 5540000], 
        'pronajem (cena bytu)': [15100, 15350],
        'prodej (cena m2)': [75800, 77000],
        'pronajem (cena m2)': [263, 265]
        })

insertion_point = 66
df_ceny = pd.concat([df_ceny.iloc[:insertion_point], lines, df_ceny.iloc[insertion_point:]]).reset_index(drop=True)


month_cz = {'leden' : '01', 'únor' : '02', 'březen' : '03', 'duben' : '04', 'květen' : '05', 'červen': '06', 
            'červenec': '07', 'srpen' : '08', 'září': '09', 'říjen' : '10', 'listopad' : '11' , 'prosinec' : '12'
           }

def change_date(datum):
    month = month_cz.get(datum.split()[0])
    year = datum.split()[1]
    day = '01'
    date = pd.to_datetime(''.join((year, month, day)), format='%Y%m%d')
    return date

df_ceny['datum'] = df_ceny['datum'].apply(change_date)

df_ceny.set_index('datum', inplace=True)
df_ceny.to_csv('vyvoj cen bytu.csv')

## 2) Výstavba nových bytů v Brně

In [46]:
# stazene csv z czso
# https://www.czso.cz/csu/czso/dokoncene-byty-v-obcich
# https://www.czso.cz/csu/xb/bytova-vystavba-v-jihomoravskem-kraji-v-1-az-2-ctvrtleti-2020

import pandas as pd
import os 

dir_path = os.getcwd() + r'\bytova vystavba'

for filename in os.listdir(dir_path):
    if filename.endswith('.csv'):
        df_tmp = pd.read_csv(os.path.join(dir_path, filename))
        df_tmp['casref_do'] = df_tmp['casref_do'].str.replace('-', '')
        df_tmp['casref_do'] = pd.to_datetime(df_tmp['casref_do'], format='%Y%m%d')
        df_tmp = df_tmp[(df_tmp['casref_do'] >= '31-12-2014') & (df_tmp['vuzemi_txt'] == 'Brno')]
        df_vystavba = df_tmp.loc[:, ['hodnota', 'casref_do']].groupby('casref_do').sum().reset_index()
        df_vystavba = df_vystavba.rename(columns={'hodnota':'pocet_dok_bytu', 'casref_do': 'datum'})
 
#doplneni radku pro rok 2020, posledni aktualni udaj za 1. pololeti z tiskove zpravy na webu czso.cz

lines = pd.DataFrame({'datum': [20200630],
        'pocet_dok_bytu': [558]
        })
lines['datum'] = pd.to_datetime(lines['datum'], format='%Y%m%d')
df_vystavba = pd.concat([df_vystavba, lines]).reset_index(drop=True)

df_vystavba.set_index('datum', inplace=True)
df_vystavba.to_csv('vystavba bytu Brno.csv')

## 3) Pohyb obyvatelstva (přirozený přírůstek, migrační saldo) v Brně 

In [47]:
# data Ceskeho statistickeho uradu, stahovani pres API https://www.api.store/

import requests
import json

url_obyv = 'https://api.apitalks.store/czso.cz/pohyb-obyvatel-za-cr-kraje-okresy-so-orp-a-obce'
filename_obyv = 'pohyb obyvatel api.csv'

def get_100rows_from_api(url, filename):   
    for i in range(0,100,20):
        query = '?filter={"where":{"rok":{"gt":2014},"vuzemi_txt":"Brno"},"limit":20,"skip":%d}' %(i)
        df_temp = get_data_from_api(url+query)
        dataframe_to_csv(df_temp, filename)
        print('inserted ' + str(i+20) + ' rows')       
        
def get_data_from_api(full_url):
    response = requests.get(full_url, headers={'x-api-key':'bTEDFRJbXBmxZe5ZE6Bb2XBuDVACUmhnhIfbqy60'} )
    data = response.json().get('data')
    df_api = pd.DataFrame.from_dict(data)
    return df_api

def dataframe_to_csv(df, filename):
    df.to_csv(filename, mode='a')
       

get_100rows_from_api(url_obyv, filename_obyv)

inserted 20 rows
inserted 40 rows
inserted 60 rows
inserted 80 rows
inserted 100 rows


In [48]:
df_obyv = pd.read_csv('pohyb obyvatel api.csv')

df_obyv.drop(df_obyv[df_obyv['idhod'] == 'idhod'].index, inplace=True)
df_obyv['casref_do'] = pd.to_datetime(df_obyv['casref_do']).apply(lambda x: x.date())
df_obyv['hodnota'] = pd.to_numeric(df_obyv['hodnota'])
df_obyv = df_obyv.loc[:, ['hodnota', 'vuk_txt', 'casref_do']].groupby(['vuk_txt', 'casref_do']).sum().reset_index()
df_obyv = df_obyv.rename(columns={'hodnota':'pocet', 'casref_do':'datum', 'vuk_txt':'kategorie'})
df_obyv = df_obyv[(df_obyv['kategorie'] == 'Přirozený přírůstek') | (df_obyv['kategorie'] == 'Migrační saldo')]
df_obyv = df_obyv.pivot(index = 'datum', columns='kategorie', values='pocet').reset_index()


#doplneni radku pro rok 2019,2020
#udaje z clanku czso na https://www.czso.cz/csu/czso/cri/pohyb-obyvatelstva-1-3-ctvrtleti-2019

new_lines = pd.DataFrame({'datum': [20191231, 20200630],
        'Migrační saldo': [75, 75 ], 
        'Přirozený přírůstek': [590, 53] 
        })
new_lines['datum'] = pd.to_datetime(new_lines['datum'], format='%Y%m%d')
df_obyv = pd.concat([df_obyv, new_lines]).reset_index(drop=True)

df_obyv.set_index('datum', inplace=True)
df_obyv.to_csv('pohyb obyvatelstva Brno.csv')

## 4) HDP a ČDDD na obyvatele v Jihomoravském kraji

In [49]:
# udaje lze zjistit jenom do roku 2018
# excelove soubory stazeny z https://www.czso.cz/csu/czso/porovnani-kraju

file_path = os.getcwd() + r'\makro data jmk\ukazatele_kraje_ekon.xlsx'

sheet_names = ['HDP_obyv', 'ČDDD_obyv']


dict_date = {'datum': ['31-12-2014', '31-12-2015', '31-12-2016', '31-12-2017', '31-12-2018']}
for sheet in sheet_names:
    tmp_df = pd.read_excel(file_path, sheet_name=sheet, index_col=1, skiprows=1)
    tmp_ser = tmp_df.loc['Jihomoravský kraj', [2014,2015,2016,2017,2018]]
    tmp_list = [round(row[1]) for row in tmp_ser.items()]
    dict_date[sheet] = tmp_list

df_makro = pd.DataFrame(dict_date)

df_makro.set_index('datum', inplace=True)
df_makro.to_csv('hdp a cddd jmk.csv')

## 5) Průměrná mzda v Brně

In [50]:
#median mezd pouze pro JMK, prumerna mzda na webu https://data.brno.cz/graf/prumerna-mzda/

df_mzda = pd.read_csv('obyvatelstvo\prum mzda.csv', delimiter=';')
df_mzda = df_mzda.iloc[6:12,0:2]
df_mzda = df_mzda.rename(columns={'Unnamed: 0':'datum', 'Brno':'prumerna mzda'})
df_mzda['datum'] = df_mzda['datum'].astype(int).astype(str) + '1231'
df_mzda['datum'] = pd.to_datetime(df_mzda['datum'].astype(str), format='%Y%m%d')
df_mzda['prumerna mzda'] = pd.to_numeric(df_mzda['prumerna mzda'])

df_mzda.set_index('datum', inplace=True)
df_mzda.to_csv('prumerna mzda Brno.csv')

## 6) Sazby hypoték

In [52]:
import requests as rq
from bs4 import BeautifulSoup as bs
import re

url_hypo = 'https://www.kurzy.cz/hypoteky/graf-sazeb-hypotek/'

def get_df_hypoteky(url):
    raw_data = scrap_data(url)
    data_in_list = rawdata_to_list(raw_data)
    df_hypo = pd.DataFrame(data_in_list[1:], columns=data_in_list[0])
    return df_hypo
    
def scrap_data(page):
    r = rq.get(page)
    soup = bs(r.text, 'html.parser')
    section = soup.find_all('script', {'type':'text/javascript'})[1]
    dataregex = re.findall("raw_data = .+", str(section))
    rawdata = dataregex[0].split('=')[1].split(']')
    return rawdata
           
def rawdata_to_list(raw_data):
    datalist = []
    for row in raw_data:
        row = row.split(',')
        new_row = []
        for i in row:
            if i != '':
                i = i.strip(" [,'")
                if 'Date' in i:
                    i = re.findall('\d{2}\s\w+\s\d{4}', i)[0]
                new_row.append(i)
        datalist.append(new_row)
    return datalist


df_hypo = get_df_hypoteky(url_hypo)

df_hypo.rename(columns={'Měsíc':'Datum', 'Průměrné sazby nových hypoték':'Prum_sazba_hypotek'}, inplace=True)
df_hypo.dropna(how='any', inplace = True)
df_hypo.drop(df_hypo.tail(1).index, inplace=True)
df_hypo['Datum'] = pd.to_datetime(df_hypo['Datum'])
df_hypo['Pribor 3M'] = pd.to_numeric(df_hypo['Pribor 3M'])
df_hypo['Prum_sazba_hypotek'] = pd.to_numeric(df_hypo['Prum_sazba_hypotek']) 
df_hypo = df_hypo[df_hypo['Datum'] >= '31-12-2014']

df_hypo.set_index('Datum', inplace=True)
df_hypo.to_csv('sazby hypotek.csv')